### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [1]:
import os
import numpy as np
from MeshPrep import convert_filtered_meshes
from MeshPrep import mesh_process_clean, string_to_array

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### 1. Cleaning the labels

The labels generated from manual curation should be processesed before passing through the geometry generation. Strongly recommended step. Generates meshes which may be used for manual cell patch selection using paraview.

NOTE: You can skip this step in case you already have the output from the *statistics collection* pipeline.

In [ ]:
# root = '../outputs/outputs_v3/output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/'

In [ ]:
# voxel_resolution = np.array([0.1625, 0.1625, 0.25])
# label_path = os.path.join(root, 'processed_labels.tif')

In [ ]:
# from VoxelProcessing import full_label_processing

# output_folder='path/folder/to/save/processed/labels'
# cell_info = full_label_processing(labeled_img=label_path, voxel_resolution=voxel_resolution, output_folder=output_folder, smoothing_iterations=10)
# label_path = os.path.join(output_folder, "processed_labels.npy")
# filtered_cell_list = cell_info[2]

#### 2. Select patch of cells for Simulations

To select smaller patches of cells for simulation one may load the labeled image in *napari* and select cells manually, or load meshes on *paraview* and use the existing tools (*suggested*). To see how to use paraview look [here](Tutorials/LabelSelection.md).

Before extracting a patch of cells, it is helpful to exclude all the cells that are cut or touch the border.

In [ ]:
from misc import get_valid_cell_ids

root = '../outputs/outputs_v3/output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/'

filtered_cell_list = get_valid_cell_ids(os.path.join(root, 'cell_stats/stats_dataset_lung_bronchiole.csv'))

In [ ]:
# The directory from which mesh files are loaded in stl format
source_mesh_path = os.path.join(root, 'cell_meshes')

# The directory in which filtered meshes will be saved in vtk format
dest_mesh_path = '../../Meshes_for_Simulation/examples/cell_clump_bronchiole' 

convert_filtered_meshes(source_mesh_path, dest_mesh_path, filtered_cell_list)

Now you can open the `.vtk` mesh files in paraview and select a clump of cells for simulation.

#### 3. Mesh Refinemnent for SimuCell3D
SimuCell3D pipeline requires rather smooth and regular meshes to work propetly.
The following cell is meant to generate and refine meshes for SimuCell3D simulation framework. 

NOTE: Ensure that the path to labels is of cleaned, processed labels. 

In [3]:
root = '/nas/groups/iber/Users/Federico_Carrara/Statistics_Collection/outputs/outputs_v5/output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/'
voxel_resolution = np.array([0.325, 0.325, 0.25])
label_path = os.path.join(root, 'processed_labels.tif')
stats_df_path = os.path.join(root, 'cell_stats/stats_dataset_intestine_villus.csv')

label_list = string_to_array("128 130 138 139 147 150 163 167 169 171 180 185 187 193 210 228")
output_dir = '/nas/groups/iber/Users/Federico_Carrara/Meshes_for_Simulation/examples/cell_clump_intestine/cell_clumps/clean_clump_16_cells/clean_meshes_v2/'

# Call the mesh_process_clean function
mesh_process_clean(
    label_path=label_path, 
    output_dir=output_dir, 
    label_list=label_list, 
    voxel_resolution=voxel_resolution,
    path_to_cell_stats_df=stats_df_path, 
    scale_factor=1e-6, 
    min_edge_length=0.5,
    make_shell=True,
    inter_meshes=False
)

-------------------------------------------
Creating meshes from labeled img...


Converting labels to meshes:  19%|█▉        | 3/16 [00:55<03:59, 18.46s/it]


KeyboardInterrupt: 

NOTE: To create a shell mesh that fits  really well the single cell meshes you may need to resort to the simulation pipeline.

In that case you need to do the following:
- In the source code (./include/mesh/cell_types), set for ECM cells `is_static=False` in the constructors, while set it to `True` for the epithelial cells.
- Set surface tension values for ECM in at the middle of the "feasible range" experimented for epithelial cells (e.g., 1e-3 could work).
- Run few simulation iterations untile the result is satisfactory. Take the resulting vtk file as the input for following simulation runs.